# Application: Creating a Telco Mobility Index

1. Add total travel distance
2. Add radius of gyration
3. Add activity entropy

In [1]:
import os
os.chdir("../")

import random
import shapely
import pendulum
import pandas as pd
from sds4gdsp.processor import convert_cel_to_point
from sds4gdsp.plotter import get_route_fig, load_images, plot_images

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
filepath_subscribers = "data/fake_subscribers.csv"
fake_subscribers = pd.read_csv(filepath_subscribers)
fake_subscribers.sample(5)

,uid,gender,age,name,chi_indicator,gcash_user_indicator
91,glo-sub-092,female,21,Barbara Brown,1,Y
67,glo-sub-068,female,45,Jodi Barron,0,N
66,glo-sub-067,male,49,Timothy Kennedy,1,N
15,glo-sub-016,female,38,Maria Rivera,1,Y
7,glo-sub-008,male,52,Timothy Perkins,1,N


In [3]:
filepath_cellsites = "data/fake_cellsites.csv"
fake_cellsites = pd.read_csv(filepath_cellsites)
fake_cellsites.sample(5)

,uid,coords
54,glo-cel-055,POINT (121.0590322 14.4964165)
26,glo-cel-027,POINT (121.0482758 14.5207167)
44,glo-cel-045,POINT (121.0560552 14.4949815)
9,glo-cel-010,POINT (121.0364372 14.5051236)
20,glo-cel-021,POINT (121.0461939 14.5245073)


In [4]:
filepath_transactions = "data/fake_transactions.csv"
fake_transactions = pd.read_csv(filepath_transactions)

In [5]:
min_date = pendulum.parse(fake_transactions.transaction_dt.min(), exact=True)
max_date = pendulum.parse(fake_transactions.transaction_dt.max(), exact=True)
period = pendulum.period(min_date, max_date)

In [6]:
sample_dt = str(random.sample(list(period), 1)[0])
filter_dt = fake_transactions.transaction_dt == sample_dt
sample_sub = fake_subscribers.uid.sample(1).item()
filter_sub = fake_transactions.sub_id == sample_sub
fake_transactions.loc[filter_sub&filter_dt]

,uid,sub_id,cel_id,transaction_dt,transaction_hr
4050,glo-txn-04051,glo-sub-012,glo-cel-015,2023-06-20,2
4051,glo-txn-04052,glo-sub-012,glo-cel-028,2023-06-20,6
4052,glo-txn-04053,glo-sub-012,glo-cel-015,2023-06-20,7
4053,glo-txn-04054,glo-sub-012,glo-cel-026,2023-06-20,8
4054,glo-txn-04055,glo-sub-012,glo-cel-033,2023-06-20,12
4055,glo-txn-04056,glo-sub-012,glo-cel-026,2023-06-20,15
4056,glo-txn-04057,glo-sub-012,glo-cel-028,2023-06-20,16
4057,glo-txn-04058,glo-sub-012,glo-cel-015,2023-06-20,18
4058,glo-txn-04059,glo-sub-012,glo-cel-026,2023-06-20,19
4059,glo-txn-04060,glo-sub-012,glo-cel-033,2023-06-20,20


In [ ]:
# day in a life of a sub
d = fake_transactions.copy()
sample_sub = d.sample(1).sub_id.item()
d = d.loc[d.sub_id==sample_sub]
days = d.transaction_dt.unique().tolist()
route_figs = []
for sample_day in days:
    sites = fake_transactions\
        .loc[fake_transactions.sub_id==sample_sub]\
        .loc[fake_transactions.transaction_dt==sample_day]\
        .cel_id.tolist()
    points = list(map(lambda z: convert_cel_to_point(z, fake_cellsites), sites))
    r = shapely.geometry.LineString(points)
    route_figs.append(get_route_fig(r))
for idx, fig in enumerate(route_figs):
    fname = "sample/{}_tmp.jpg".format(idx+1)
    fig.savefig(fname)
imgs = load_images("sample")
plot_images(imgs)